In [ ]:
# %% Minimal setup
# If needed (uncomment in a notebook):
# !pip install requests python-dotenv

import os, json, textwrap, re, time
import requests

API_KEY  = os.getenv("OPENAI_API_KEY", "cse476")
API_BASE = os.getenv("API_BASE", "http://10.4.58.53:41701/v1")  
MODEL    = os.getenv("MODEL_NAME", "bens_model")              

def call_model_chat_completions(prompt: str,
                                system: str = "You are a helpful assistant. Reply with only the final answer—no explanation.",
                                model: str = MODEL,
                                temperature: float = 0.3,
                                timeout: int = 60,
                                max_tokens: int = 128) -> dict:
    """
    Calls an OpenAI-style /v1/chat/completions endpoint and returns:
    { 'ok': bool, 'text': str or None, 'raw': dict or None, 'status': int, 'error': str or None, 'headers': dict }
    """
    url = f"{API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type":  "application/json",
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user",   "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
    }

    try:
        #{'id': 'chatcmpl-88b6d7e18a5542b5bed5bf2828f0661e', 'object': 'chat.completion', 'created': 1763204718, 'model': 'bens_model', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'US Highway 281', 'refusal': None, 'annotations': None, 'audio': None, 'function_call': None, 'tool_calls': [], 'reasoning_content': None}, 'logprobs': None, 'finish_reason': 'stop', 'stop_reason': None, 'token_ids': None}], 'service_tier': None, 'system_fingerprint': None, 'usage': {'prompt_tokens': 50, 'total_tokens': 57, 'completion_tokens': 7, 'prompt_tokens_details': None}, 'prompt_logprobs': None, 'prompt_token_ids': None, 'kv_transfer_params': None}
        resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
        status = resp.status_code
        hdrs   = dict(resp.headers)
        if status == 200:
            data = resp.json()
            #print(data)
            text = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            tokens_used = data.get("usage",[{}]).get("completion_tokens", {})
            #print('used tokens:', tokens_used)
            
            return {"ok": True, "text": text, "raw": data, "status": status, "error": None, "headers": hdrs, "tokens_used":tokens_used}
        else:
            # try best-effort to surface error text
            err_text = None
            try:
                err_text = resp.json()
            except Exception:
                err_text = resp.text
            return {"ok": False, "text": None, "raw": None, "status": status, "error": str(err_text), "headers": hdrs}
    except requests.RequestException as e:
        return {"ok": False, "text": None, "raw": None, "status": -1, "error": str(e), "headers": {}}


In [107]:
from IPython.display import Markdown, display

In [ ]:
# %% Direct call example
def direct_call(prompt="What is 17 + 28? Answer with just the number.", temperature=0.2, max_tokens=128):
    demo_prompt = prompt
    result = call_model_chat_completions(demo_prompt, temperature=temperature, max_tokens=max_tokens)
    print("OK:", result["ok"], "HTTP:", result["status"])
    print("MODEL SAYS:", (result["text"] or "").strip())

    # Optional: Inspect rate-limit headers if your provider exposes them
    for k in ["x-ratelimit-remaining-requests", "x-ratelimit-limit-requests", "x-request-id"]:
        if k in result["headers"]:
            print(f"{k}: {result['headers'][k]}")


In [90]:
# %% Define three tests: input + expected
my_tests = [
    {
        "id": "math_inequality",
        "type": "numeric",  # grader will prefer numeric extraction
        "prompt": "Solve for the smallest integer n such that 3n + 5 > 26. Answer with just the integer.",
        "expected": "8",    # Because 3n > 21 => n > 7, smallest integer is 8
    },
    {
        "id": "commonsense_ice",
        "type": "text",
        "prompt": (
            "You place an ice cube in a glass of water and mark the water level. "
            "After the ice melts, does the water level rise, fall, or stay the same? "
            "Answer with exactly one of: 'rise', 'fall', 'stay the same'."
        ),
        "expected": "stay the same",
    },
    {
        "id": "logic_race",
        "type": "text",
        "prompt": (
            "In a race, you pass the person in second place. What position are you now in? "
            "Answer with a single word like 'first', 'second', 'third'."
        ),
        "expected": "second",
    },
]


In [142]:
import json
from pprint import pprint
import random
from collections import Counter

POSSIBLE_TYPES = ['math', 'common_sense', 'planning', 'coding', 'future_prediction']

all_tests = json.load(open("parsed_dev_data.json", "r", encoding="utf-8"))

type_counts = Counter(t['domain'] for t in all_tests)
print(type_counts)

formatted_tests = []
for i, t in enumerate(all_tests, start=1):
    
    formatted_tests.append({
        "id": t['id'], # domain_domainIndex_domainTestIndex_testIndex
        "type": t['domain'],
        "prompt": t['input'],
        "expected": t['output'],
        "char_count": t['input_char_count'],
        "exp_word_count": t['exp_word_count']
    })
    
all_tests = formatted_tests

Counter({'common_sense': 400, 'math': 300, 'coding': 100, 'future_prediction': 100, 'planning': 100})


In [298]:
def print_test(test):
    print(json.dumps(test, indent=2, ensure_ascii=False))

#pass test_type as a list of types
#generalized get test function
def get_tests(n=0, test_type=POSSIBLE_TYPES, start=0, end=None, lower_char=0, upper_char=float('inf'), lower_exp=0, upper_exp=float('inf'), seed=None):
    filtered_tests = [t for t in all_tests if t['type'] in test_type and lower_char <= t['char_count'] <= upper_char and lower_exp <= t['exp_word_count'] <= upper_exp]
    print('filtered size:', len(filtered_tests))
    sample_size = min(n, len(filtered_tests))
    
    if seed is not None: random.seed(seed)
    
    if n == 0:
        return filtered_tests[start:end]
    elif n == -1:
        filtered_type_counts = Counter(t['type'] for t in filtered_tests)
        each_test = []
        count = 0
        
        for val in filtered_type_counts.values():
            rand = random.randint(count, count + val)
            count = count + val
            each_test.append(filtered_tests[rand])
            
        print("sampled size:", len(each_test))    
        return each_test
    else:
        return random.sample(filtered_tests, sample_size)
    
""" def get_test_type(test_type, start=0, end=None, lower=0, upper=float('inf')):
    tests = [t for t in all_tests if t['type'] in test_type and lower <= t['char_count'] <= upper]
    return tests[start:end]

def get_random_tests(n=5, lower=0, upper=float('inf'), test_type=POSSIBLE_TYPES):
    filtered_tests = get_test_type(test_type=test_type, lower=lower, upper=upper) #[t for t in all_tests if lower <= t['char_count'] <= upper]
    sample_size = min(n, len(filtered_tests)) #prevent error
    return random.sample(filtered_tests, sample_size) """

" def get_test_type(test_type, start=0, end=None, lower=0, upper=float('inf')):\n    tests = [t for t in all_tests if t['type'] in test_type and lower <= t['char_count'] <= upper]\n    return tests[start:end]\n\ndef get_random_tests(n=5, lower=0, upper=float('inf'), test_type=POSSIBLE_TYPES):\n    filtered_tests = get_test_type(test_type=test_type, lower=lower, upper=upper) #[t for t in all_tests if lower <= t['char_count'] <= upper]\n    sample_size = min(n, len(filtered_tests)) #prevent error\n    return random.sample(filtered_tests, sample_size) "

In [ ]:
tests = get_tests(n=1, upper_char=300) #get_test_type('math', end=10, lower=0, upper=500)
pprint(tests)

filtered size: 440
['First',
 'figure',
 'out',
 'how',
 'many',
 'square',
 'feet',
 'the',
 'original',
 'bolt',
 'of',
 'fabric',
 'was:',
 '16',
 'feet',
 '*',
 '12',
 'feet',
 '=',
 '<<16*12=192>>192',
 'square',
 'feet',
 'Then',
 'figure',
 'out',
 'how',
 'much',
 'fabric',
 'Ann',
 'took',
 'for',
 'the',
 'living',
 'room',
 'curtains:',
 '4',
 'feet',
 '*',
 '6',
 'feet',
 '=',
 '<<4*6=24>>24',
 'square',
 'feet',
 'Then',
 'figure',
 'out',
 'how',
 'much',
 'fabric',
 'Ann',
 'took',
 'for',
 'the',
 'bathroom',
 'curtains:',
 '2',
 'feet',
 '*',
 '4',
 'feet',
 '=',
 '<<2*4=8>>8',
 'square',
 'feet',
 'Finally,',
 'subtract',
 'the',
 'square',
 'footage',
 'of',
 'both',
 'sets',
 'of',
 'curtains',
 'from',
 'the',
 'total',
 'square',
 'footage:',
 '192',
 '-',
 '24',
 '-',
 '8',
 '=',
 '<<192-24-8=160>>160',
 'square',
 'feet',
 '####',
 '160']


In [94]:
#simple hello world call to kick off the commits
#direct_call(prompt="how do I find the derivative of y=x^2 using python?")

In [95]:
def interactive_chat():
    messages = ["<Start of message history>"]
    count = 0
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit']:
            print("Exiting chat.")
            break
        response = call_model_chat_completions(prompt=f"Old messages{messages}, CURRENT USER INPUT:{user_input} <--- ANSWER THIS QUESTION", temperature=0.7)
        count += 1
        messages.append(f"MESSAGE_{count}_[previous user input: {user_input}, previous system response: {response['text']}]")
        if response["ok"]:
            print("Model:", response["text"].strip())
        else:
            print("Error:", response["error"])
        print(messages)
#interactive_chat()

In [ ]:
""" def execute_tests():
    rows = []
    for t in tests:
        r = call_model_chat_completions(
            prompt,
            system=system,
            model=model,
            temperature=0.3,
            max_tokens=128
        ) """

' def execute_tests():\n    rows = []\n    for t in tests:\n        r = call_model_chat_completions(\n            prompt,\n            system=system,\n            model=model,\n            temperature=0.3,\n            max_tokens=128\n        ) '

In [198]:
def self_evaluate(question, prediction, expected_answer, model=MODEL):
    """
    Use the model itself as a strict grader.
    Returns True if the model says the prediction matches the expected answer; else False.
    Falls back to a simple normalized string compare if the model's reply is malformed.
    """
    import re

    system = "You are a strict grader. Reply with exactly True or False. No punctuation. No explanation."
    prompt = f"""You are grading a question-answer pair.

Return exactly True if the PREDICTION would be accepted as correct for the EXPECTED_ANSWER.
Otherwise, return False.

QUESTION:
{question}

PREDICTION:
{prediction}

EXPECTED_ANSWER:
{expected_answer}

Answer with exactly: True or False
"""

    r = call_model_chat_completions(
        prompt,
        system=system,
        model=model,
        temperature=0.3,
    )

    reply = (r.get("text") or "").strip().lower()
    if reply.startswith("true"):
        return True
    if reply.startswith("false"):
        return False

    # No Fallback yet


In [272]:
def self_evaluate2(question, model_output, prediction, expected_answer, model=MODEL):
    """
    Use the model itself as a strict grader.
    Returns True if the model says the prediction matches the expected answer; else False.
    Falls back to a simple normalized string compare if the model's reply is malformed.
    """
    import re

    system = "You are a strict grader. Reply with exactly Yes or No. No punctuation. No explanation."
    prompt = f"""MODEL_1 thinks this ANSWER is {prediction}, do you agree with MODEL_1 decision?

QUESTION:
{question}

ANSWER:
{model_output}

EXPECTED_ANSWER:
{expected_answer}

-----------------------
MODEL_1 OUTPUT:
{prediction}
-----------------------

Answer with exactly: Yes or No. Do you agree with MODEL_1?
"""

    r = call_model_chat_completions(
        prompt,
        system=system,
        model=model,
        temperature=0.3,
    )

    reply = (r.get("text") or "").strip().lower()
    if reply.startswith("true") or reply.startswith("yes"):
        return True
    if reply.startswith("false") or reply.startswith("no"):
        return False

    # No Fallback yet


In [296]:
tf_map = {'yes':'true', 'no':'false'}
def map_tf(output, exp):
    exp = exp.lower().strip('.')
    out = output.lower().strip('.')
    
    #rare case when exp is actually yes/now and model output is true/false
    if exp == "yes" and out == "true": return "yes"
    if exp == "no" and out == "false": return "no"
    
    return tf_map.get(out) if out in tf_map else output

In [303]:

def basic_match_check(test, output):
    exp = test["expected"]
    
    output = map_tf(output, exp)
    
    matches = re.findall(re.escape(str(exp)), output, re.IGNORECASE)
    
    num_matches = len(matches)
    if num_matches > 0:
        #print('MATCH(ES) FOUND:', matches)
        return True
    
    #print('NO MATCH FOUND')
    return False

In [279]:
def seperator(text, tokens_used=None, max_tokens=400):
    if tokens_used is not None:
        print(f'{text} (TOKENS USED: {tokens_used}/{max_tokens})')
        if int(tokens_used) == max_tokens:
            print('MAXED TOKENS REACHED - OUTPUT TRUNCATED')
            return False
    else:
        print(text)
    print('-'*32)
    
    return True

In [273]:
def check_correct(bool1, bool2):
    correctness = bool1 and bool2
    agreement = bool1 == bool2
    
    print('✅ CORRECT') if correctness else print('❌ INCORRECT')
    print('🆗 AGREED') if agreement else print('🆘 DISAGREED')
    
    return correctness, agreement

In [ ]:
import math

def create_matches(toCount, toMatch):
    counter = Counter(toCount)
    match_counts = {word: counter.get(word, 0) for word in toMatch}
    total_matches = sum(match_counts.values())
    output_len = len(toCount)
    print(f"{total_matches}/{output_len} : {(total_matches / output_len) * 100 if output_len != 0 else 0}%")
    print('match counts:', match_counts)
    return total_matches, output_len

def get_cosine(expected_counter, output_counter):
    dot_product = sum(expected_counter[word] * output_counter.get(word, 0) for word in expected_counter)
    
    print(f"Dot product: {dot_product}")
    
    exp_mag = math.sqrt(sum(v**2 for v in expected_counter.values()))
    out_mag = math.sqrt(sum(v**2 for v in output_counter.values()))
    
    if exp_mag > 0 and out_mag > 0:
        cosine_sim = dot_product / (exp_mag * out_mag)
        print(f"Cosine similarity: {cosine_sim}")
        
    return cosine_sim

def super_match(test, output):
    expected = test["expected"].lower().split()
    output = output.lower().split()
    
    expected_counter = Counter(expected)
    output_counter = Counter(output)
    
    #not very helpful in the long run...
    get_cosine(expected_counter, output_counter)
    
    exp_matches, exp_len = create_matches(output, expected)
    out_matches, out_len = create_matches(expected, output)
    
    #return match_counts

In [304]:
def self_evaluate_tests(tests, model=MODEL, grader_model=None, sleep_sec=0.2, verbose=True):
    """
    Run the tests by querying the model for each prompt, then use LLM-as-a-judge
    (self_evaluate) to determine correctness.

    Args:
        tests: list of dicts with keys: id, prompt, expected (and optionally type)
        model: model used to generate predictions
        grader_model: model used to judge correctness (defaults to `model` if None)
        sleep_sec: small delay between calls to be polite to the API
        verbose: if True, print a summary line per test

    Returns:
        rows: list of dicts with fields:
              id, expected, got, correct, status, error
    """
    import time

    judge_model = grader_model or model
    MAX_TOKENS = 400
    final_answers = []
    count = 0
    
    for t in tests:
        count += 1
        # 1) Get model prediction
        #print('prompt:', t['prompt'])
        print('\n','='*64)
        seperator('TEST_CASE')
        print_test(t)
        r = call_model_chat_completions(
            f"{t['prompt']}",
            system="Give a short answer to each prompt, don't explain.",
            model=model,
            temperature=0.3,
            max_tokens=MAX_TOKENS
        )
        got = (r.get("text") or "").strip()
        tokens_used = r.get("tokens_used")
        

        got = map_tf(got, t["expected"])
        
        #If output is truncated and both evals return true, return false
        not_truncated = seperator('\nMODEL_OUTPUT', tokens_used, MAX_TOKENS)
        display(Markdown(f"\n{got}"))
        #print('raw: ', got)
        
        super_match(t, got)
        match_check = basic_match_check(t, got)
        match_check = bool(match_check)
        
        # 2) LLM-as-a-judge: strict True/False
        is_correct = self_evaluate(
            question=t["prompt"],
            prediction=got,
            expected_answer=t["expected"],
            model=judge_model,
        )
        is_correct = bool(is_correct)
        
        seperator('\nEVALUATION')
        print('match check:', match_check)
        print('self_eval:', is_correct)
        correctness, agreement = check_correct(match_check, is_correct)
        
        if not agreement:
            seperator('\nDISAGREEMENT --> SECOND EVAL')
            is_correct2 = self_evaluate2(
                question=t["prompt"],
                model_output=got,
                expected_answer=t["expected"],
                prediction=is_correct,
                model=judge_model
            )
            is_correct2 = bool(is_correct2)
            
            print('self_eval2:', is_correct2)
            correctness, agreement = check_correct(is_correct, is_correct2)
            
            if not not_truncated and correctness:
                correctness = False
                print("❌ INCORRECT | BOTH EVALS RETURNED TRUE BUT OUTPUT WAS TRUNCATED")

        final_answers.append(correctness)
        
        if sleep_sec:
            time.sleep(sleep_sec)

    return final_answers

# Example:


In [120]:
import re

In [307]:
test_prompts = get_tests(n=5, seed=42) #get_test_type(["math"],end=10, upper=300) get_random_tests(n=3, upper=300)
results_llm_judge = self_evaluate_tests(test_prompts, verbose=True, model=MODEL, grader_model=MODEL)
print("\n","="*64)
print(results_llm_judge)

filtered size: 1000

TEST_CASE
--------------------------------
{
  "id": "math_3_55_655",
  "type": "math",
  "prompt": "Nancy is filling an aquarium for her fish. She fills it halfway and goes to answer the door. While she's gone, her cat knocks the aquarium over and spills half the water in it. Then Nancy comes back and triples the amount of water in the aquarium. If the aquarium is 4 feet long, 6 feet wide, and 3 feet high, how many cubic feet of water are in the aquarium?",
  "expected": "First calculate the volume of the aquarium by multiplying its length, width and height: 4 ft * 6 ft * 3 ft = <<4*6*3=72>>72 cubic ft\nThen figure out what proportion of the aquarium is full after the cat knocks it over: 1/2 * 1/2 = 1/4\nThen figure out what proportion of the aquarium is full after Nancy refills it: 3 * 1/4 = 3/4\nNow multiply the proportion of the aquarium that's full by the aquarium's volume to find out how much water is in it: 72 cubic ft * 3/4 = <<72*3/4=54>>54 cubic ft\n#### 


First, calculate the total volume of the aquarium:  
4 × 6 × 3 = 72 cubic feet  

Nancy fills it halfway:  
72 ÷ 2 = 36 cubic feet  

The cat spills half the water:  
36 ÷ 2 = 18 cubic feet  

Nancy triples the amount:  
18 × 3 = 54 cubic feet  

**Answer: 54**

27/54 : 50.0%
match counts: {'first': 0, 'calculate': 1, 'the': 5, 'volume': 1, 'of': 1, 'aquarium': 0, 'by': 0, 'multiplying': 0, 'its': 0, 'length,': 0, 'width': 0, 'and': 0, 'height:': 0, '4': 1, 'ft': 0, '*': 0, '6': 1, '3': 2, '=': 4, '<<4*6*3=72>>72': 0, 'cubic': 4, 'then': 0, 'figure': 0, 'out': 0, 'what': 0, 'proportion': 0, 'is': 0, 'full': 0, 'after': 0, 'cat': 1, 'knocks': 0, 'it': 1, 'over:': 0, '1/2': 0, '1/4': 0, 'nancy': 2, 'refills': 0, 'it:': 0, '3/4': 0, 'now': 0, 'multiply': 0, "that's": 0, "aquarium's": 0, 'to': 0, 'find': 0, 'how': 0, 'much': 0, 'water': 0, 'in': 0, '72': 2, '<<72*3/4=54>>54': 0, '####': 0, '54': 1}
31/99 : 31.313131313131315%
match counts: {'first,': 0, 'calculate': 1, 'the': 8, 'total': 0, 'volume': 2, 'of': 4, 'aquarium:': 0, '4': 1, '×': 0, '6': 1, '3': 2, '=': 4, '72': 1, 'cubic': 3, 'feet': 0, 'nancy': 1, 'fills': 0, 'it': 1, 'halfway:': 0, '÷': 0, '2': 0, '36': 0, 'cat': 1, 'spills': 0, 'half': 0, 'water:': 0, '18': 0, 'triples': 0, 'amount:

0/0 : 0%
match counts: {'edward': 0}
0/1 : 0.0%
match counts: {}
Dot product: 0

EVALUATION
--------------------------------
match check: False
self_eval: False
❌ INCORRECT
🆗 AGREED

TEST_CASE
--------------------------------
{
  "id": "coding_0_26_26",
  "type": "coding",
  "prompt": "Create a Pandas DataFrame from a list of tuples, each representing a row. Tuples of unequal lengths are allowed, and missing elements are filled with None. Optionally, missing numeric values can be filled with random data. >>> data = [('Mango', 20), ('Apple', ), ('Banana', )] >>> df = task_func(data, columns=['Fruit', 'Quantity'], fill_missing=False, seed=42) >>> print(df) Fruit  Quantity 0   Mango      20.0 1   Apple       NaN 2  Banana       NaN\nThe function should output with:\n    DataFrame: A pandas DataFrame with specified columns.\n    Missing elements are represented as None or filled with random data.\nYou should write self-contained code starting with:\n```\nimport pandas as pd\nimport random\


```python
import pandas as pd
import random

def task_func(data, columns=['Name', 'Age', 'Occupation'], fill_missing=False, num_range=(0, 100), seed=None):
    if seed is not None:
        random.seed(seed)
    df = pd.DataFrame(data, columns=columns)
    if fill_missing:
        for col in df.columns:
            if df[col].dtype in (float, int):
                df[col] = df[col].fillna(random.randint(*num_range))
    return df
```

23/43 : 53.48837209302325%
match counts: {'if': 3, 'seed': 1, 'is': 1, 'not': 1, 'none:': 1, 'random.seed(seed)': 1, 'df': 2, '=': 2, 'pd.dataframe(data,': 1, 'columns=columns)': 1, 'fill_missing:': 1, 'for': 1, 'col': 1, 'in': 2, 'df.columns:': 1, 'df[col].dtype': 1, "['float64',": 0, "'int64']:": 0, 'df[col]': 1, 'df[col].apply(lambda': 0, 'x:': 0, 'random.randint(*num_range)': 0, 'pd.isnull(x)': 0, 'else': 0, 'x)': 0, 'return': 1}
24/32 : 75.0%
match counts: {'```python': 0, 'import': 0, 'pandas': 0, 'as': 0, 'pd': 0, 'random': 0, 'def': 0, 'task_func(data,': 0, "columns=['name',": 0, "'age',": 0, "'occupation'],": 0, 'fill_missing=false,': 0, 'num_range=(0,': 0, '100),': 0, 'seed=none):': 0, 'if': 4, 'seed': 1, 'is': 1, 'not': 1, 'none:': 1, 'random.seed(seed)': 1, 'df': 2, '=': 2, 'pd.dataframe(data,': 1, 'columns=columns)': 1, 'fill_missing:': 1, 'for': 1, 'col': 1, 'in': 2, 'df.columns:': 1, 'df[col].dtype': 1, '(float,': 0, 'int):': 0, 'df[col]': 1, 'df[col].fillna(random.randi


First, fix one person to remove rotational symmetry, say fix Pierre. Then arrange the remaining 6 people.

We need to ensure Rosa and Thomas are not adjacent to Pierre. So, we place the other 4 people first, creating 4 gaps. Choose 2 gaps for Rosa and Thomas, and arrange them: $ \binom{4}{2} \times 2! = 12 $.

Then arrange the remaining 4 people in the remaining 4 seats: $ 4! = 24 $.

Total: $ 12 \times 24 = 288 $.

Answer: $ \boxed{288} $

0/85 : 0.0%
match counts: {'144': 0}
0/1 : 0.0%
match counts: {'first,': 0, 'fix': 0, 'one': 0, 'person': 0, 'to': 0, 'remove': 0, 'rotational': 0, 'symmetry,': 0, 'say': 0, 'pierre.': 0, 'then': 0, 'arrange': 0, 'the': 0, 'remaining': 0, '6': 0, 'people.': 0, 'we': 0, 'need': 0, 'ensure': 0, 'rosa': 0, 'and': 0, 'thomas': 0, 'are': 0, 'not': 0, 'adjacent': 0, 'so,': 0, 'place': 0, 'other': 0, '4': 0, 'people': 0, 'creating': 0, 'gaps.': 0, 'choose': 0, '2': 0, 'gaps': 0, 'for': 0, 'thomas,': 0, 'them:': 0, '$': 0, '\\binom{4}{2}': 0, '\\times': 0, '2!': 0, '=': 0, '12': 0, '$.': 0, 'in': 0, 'seats:': 0, '4!': 0, '24': 0, 'total:': 0, '288': 0, 'answer:': 0, '\\boxed{288}': 0}
Dot product: 0
Cosine similarity: 0.0

EVALUATION
--------------------------------
match check: False
self_eval: False
❌ INCORRECT
🆗 AGREED

TEST_CASE
--------------------------------
{
  "id": "common_sense_1_182_282",
  "type": "common_sense",
  "prompt": "Still Da Baddest is the fourth studio album by American


Erykah Badu

0/2 : 0.0%
match counts: {'keyshia': 0, 'cole': 0}
0/2 : 0.0%
match counts: {'erykah': 0, 'badu': 0}
Dot product: 0
Cosine similarity: 0.0

EVALUATION
--------------------------------
match check: False
self_eval: False
❌ INCORRECT
🆗 AGREED

[True, False, False, False, False]
